In [76]:
#데이터 불러오기
import pandas as pd
df=pd.read_csv('data/df_top15.csv')

In [77]:
df

,년,월,국적지역,입국자수,날짜
0,2018,1,대만,78321,2018-01-01
1,2018,2,대만,90673,2018-02-01
2,2018,3,대만,96811,2018-03-01
3,2018,4,대만,107719,2018-04-01
4,2018,5,대만,91102,2018-05-01
...,...,...,...,...,...
712,2024,8,홍콩,61592,2024-08-01
713,2024,9,홍콩,39815,2024-09-01
714,2024,10,홍콩,52005,2024-10-01
715,2024,11,홍콩,46366,2024-11-01


In [80]:
# 원핫 인코딩 적용
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)  # 최신 버전에서는 sparse_output 사용
encoded_countries = encoder.fit_transform(df[['국적지역']])

encoded_df = pd.DataFrame(encoded_countries, columns=encoder.get_feature_names_out(['국적지역']))

# 기존 데이터프레임과 병합
df_encoded = pd.concat([df[['년', '월', '입국자수']], encoded_df], axis=1)

In [81]:
df_encoded

,년,월,입국자수,국적지역_러시아,국적지역_말레이시아,국적지역_미국,국적지역_베트남,국적지역_싱가포르,국적지역_영국,국적지역_오스트레일리아,국적지역_인도네시아,국적지역_일본,국적지역_중국,국적지역_캐나다,국적지역_태국,국적지역_필리핀,국적지역_홍콩
0,2018,1,78321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018,2,90673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018,3,96811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018,4,107719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018,5,91102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2024,8,61592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
713,2024,9,39815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
714,2024,10,52005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
715,2024,11,46366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [82]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LinearRegression

# 독립 변수(X)와 종속 변수(y) 설정
X = df_encoded.drop(columns=['입국자수'])
y = df_encoded['입국자수']



In [83]:
#스케일링은 제외 : 데이터의 해석 가능성 유지

#트레이닝, 테스트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

#모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [84]:
# 예측 수행
y_pred = model.predict(X_test)

In [85]:
#성능평가
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)


0.8870442044329196

In [86]:
# 2025년, 2026년 데이터를 생성하여 예측
future_years = [2025, 2026]
future_months = list(range(1, 13))  # 1월~12월
# 기존 데이터에서 국적 리스트 추출
countries = encoder.get_feature_names_out(['국적지역'])

In [90]:
countries.shape

(14,)

In [98]:
# 예측을 위한 데이터 생성
future_data = []
for year in future_years:
    for month in future_months:
        for country in countries:
            # 국가별 원핫 인코딩 값 생성
            country_encoding = [1 if c == country else 0 for c in countries]
            future_data.append([year, month] + country_encoding)

# 데이터프레임 변환
future_df = pd.DataFrame(future_data, columns=['년', '월'] + list(countries))

# 선형 회귀 모델을 사용하여 미래 입국자 수 예측
future_predictions = model.predict(future_df)

# 결과 데이터프레임 생성
future_df['예측 입국자수'] = future_predictions

#각 국적지역-국적 컬럼을 원래처럼 변환
future_df['국적지역'] = future_df[countries].idxmax(axis=1)

# 년, 월, 국적지역, 예측 입국자수 컬럼만 선택
future_df = future_df[['년', '월', '국적지역', '예측 입국자수']]
#국적지역_을 제거
future_df['국적지역'] = future_df['국적지역'].str.replace('국적지역_', '')
#국적지역별로 정렬
future_df = future_df.sort_values(by=['국적지역', '년', '월'])
#예측입국자수 정수로 변환
future_df['예측 입국자수'] = future_df['예측 입국자수'].astype(int).reset_index(drop=True)

future_df

,년,월,국적지역,예측 입국자수
0,2025,1,러시아,1311
14,2025,2,러시아,3140
28,2025,3,러시아,18025
42,2025,4,러시아,19854
56,2025,5,러시아,92299
...,...,...,...,...
279,2026,8,홍콩,23651
293,2026,9,홍콩,28890
307,2026,10,홍콩,30719
321,2026,11,홍콩,47385


In [99]:
future_df[future_df['국적지역']=='미국']

,년,월,국적지역,예측 입국자수
2,2025,1,미국,5397
16,2025,2,미국,7226
30,2025,3,미국,22112
44,2025,4,미국,23941
58,2025,5,미국,96386
72,2025,6,미국,26547
86,2025,7,미국,28376
100,2025,8,미국,13361
114,2025,9,미국,15190
128,2025,10,미국,8622


In [101]:
df[df['국적지역']=='미국']

,년,월,국적지역,입국자수,날짜
36,2018,1,미국,68071,2018-01-01
37,2018,2,미국,76797,2018-02-01
38,2018,3,미국,87866,2018-03-01
39,2018,4,미국,100670,2018-04-01
40,2018,5,미국,92902,2018-05-01
41,2018,6,미국,102403,2018-06-01
42,2018,7,미국,98111,2018-07-01
43,2018,8,미국,84466,2018-08-01
44,2018,9,미국,85895,2018-09-01
45,2018,10,미국,108108,2018-10-01


In [100]:
# 점수는 높았지만 가중치가 음수가 되면서 예측값이 마이너스가 나오는 경우가 있었음